# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.5 gym keras-rl2 gym[atari]

# 1. Test Random Environment with OpenAI Gym

In [1]:
import gym 
import random
import retro
import numpy as np

In [6]:
env = retro.make(game='Airstriker-Genesis',use_restricted_actions=retro.Actions.DISCRETE)
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [7]:
env = env.unwrapped

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        n_state, reward, done, info = env.step(env.action_space.sample())
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:160.0
Episode:2 Score:20.0
Episode:3 Score:120.0
Episode:4 Score:80.0
Episode:5 Score:60.0


In [7]:
env.action_space.sample()

48

# 2. Create a Deep Learning Model with Keras

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D,MaxPool3D
from tensorflow.keras.optimizers import Adam

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(MaxPool3D(pool_size=(1, 2, 2), strides=None, padding='valid'))
    model.add(Convolution2D(16, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
del model

In [17]:
model = build_model(height, width, channels, actions)

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 55, 79, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 26, 38, 64)     32832     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 3, 13, 19, 64)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 11, 17, 16)     9232      
_________________________________________________________________
flatten_1 (Flatten)          (None, 8976)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               4596224   
_________________________________________________________________
dense_5 (Dense)              (None, 256)              

# 3. Build Agent with Keras-RL

In [19]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [20]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [21]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

/home/seb/.local/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [22]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [23]:
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)

Training for 100000 steps ...
Interval 1 (0 steps performed)


/home/seb/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 1053/10000 [==>...........................] - ETA: 11:17 - reward: 0.0000e+00done, took 81.402 seconds


In [24]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 80.000, steps: 2563


KeyboardInterrupt: 

# 4. Reloading Agent from Memory

In [21]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [22]:
del model, dqn

In [26]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')